In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import sklearn.metrics as metrics
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16

In [ ]:
from glob import glob # global directory to access images

In [ ]:
num_classes=2
IMAGE_SHAPE = [224, 224]
batch_size=32
epochs = 5

In [ ]:
vgg = VGG16(input_shape = (224,224,3), weights = 'imagenet', include_top = False)
for layer in vgg.layers:
 layer.trainable = False
x = Flatten()(vgg.output)
x = Dense(128, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(num_classes, activation = 'softmax')(x)
model = Model(inputs = vgg.input, outputs = x)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
trdata = ImageDataGenerator()
train_data_gen = trdata.flow_from_directory(directory="/content/drive/MyDrive/NNDL/Train",target_size=(224,224), shuffle=False, class_mode='categorical')
tsdata = ImageDataGenerator()
test_data_gen = tsdata.flow_from_directory(directory="/content/drive/MyDrive/NNDL/Test", target_size=(224,224),shuffle=False, class_mode='categorical')

Found 125 images belonging to 2 classes.
Found 25 images belonging to 2 classes.


In [ ]:
training_steps_per_epoch = np.ceil(train_data_gen.samples / batch_size)
validation_steps_per_epoch = np.ceil(test_data_gen.samples / batch_size)
model.fit_generator(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=test_data_gen, validation_steps=validation_steps_per_epoch,epochs=epochs, verbose=1)
print('Training Completed!')

<ipython-input-6-b050a153b951>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data_gen, steps_per_epoch = training_steps_per_epoch, validation_data=test_data_gen, validation_steps=validation_steps_per_epoch,epochs=epochs, verbose=1)


Epoch 1/5
4/4 [==============================] - 47s 12s/step - loss: 34.4895 - accuracy: 0.8560 - val_loss: 34.4392 - val_accuracy: 0.4000
Epoch 2/5
4/4 [==============================] - 1s 227ms/step - loss: 28.8816 - accuracy: 0.4240 - val_loss: 5.4815 - val_accuracy: 0.8000
Epoch 3/5
4/4 [==============================] - 1s 243ms/step - loss: 0.8216 - accuracy: 0.9520 - val_loss: 1.7920 - val_accuracy: 0.8400
Epoch 4/5
4/4 [==============================] - 1s 264ms/step - loss: 3.8133 - accuracy: 0.8240 - val_loss: 2.5132 - val_accuracy: 0.8400
Epoch 5/5
4/4 [==============================] - 1s 247ms/step - loss: 0.4195 - accuracy: 0.9680 - val_loss: 8.5235 - val_accuracy: 0.7600
Training Completed!


In [ ]:
Y_pred = model.predict(test_data_gen, test_data_gen.samples / batch_size)
val_preds = np.argmax(Y_pred, axis=1)
import sklearn.metrics as metrics
val_trues =test_data_gen.classes
from sklearn.metrics import classification_report
print(classification_report(val_trues, val_preds))

1/1 [==============================] - 0s 283ms/step
              precision    recall  f1-score   support

           0       0.71      1.00      0.83        15
           1       1.00      0.40      0.57        10

    accuracy                           0.76        25
   macro avg       0.86      0.70      0.70        25
weighted avg       0.83      0.76      0.73        25



In [ ]:
keras_file="Model.h5"
tf.keras.models.save_model(model,keras_file)

<ipython-input-8-981ad33596cb>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model,keras_file)


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
model = load_model('Model.h5')
img_path = '/content/drive/MyDrive/cat.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds=model.predict(x)
# create a list containing the class labels
class_labels = ['Cat','Dog']
# find the index of the class with maximum score
pred = np.argmax(preds, axis=-1)
# print the label of the class with maximum score
print(class_labels[pred[0]])

1/1 [==============================] - 1s 1s/step
Cat
